In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'd:\\python_project\\DataScience\\StoresSalesPrediction\\BigMartStoreSalesPrediction'

In [4]:
#create data validation entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    unzip_data_dir: Path
    STATUS_FILE: str
    STATUS_DATA_TYPE_FILE: str
    all_schema: dict
    target_variable: dict 

In [5]:
from salesPrediction.constants import *
from salesPrediction.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS
        target_variable = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            STATUS_DATA_TYPE_FILE = config.STATUS_DATA_TYPE_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
            target_variable = target_variable
        )

        return data_validation_config

In [7]:
import os
from salesPrediction import logger
import pandas as pd
import json

In [ ]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, "w") as f:
                        f.write(f"Validation Status col {col}: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, "w") as f:
                        f.write(f"Validation Status col {col}: {validation_status}")

            return validation_status
        except Exception as e:
            raise e

    def validate_column_datatype(self) -> dict:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)

            all_schema = self.config.all_schema
            mismatched_cols = {}
            
            for col in all_schema:
                if data[col].dtype != all_schema[col]:
                    
                    validation_status = "DATATYPE_MISMATCH"

                    actual_type = str(data[col].dtype)
                    expected_type = all_schema[col]

                    mismatched_cols[col] = {"expected" : expected_type, "actual" : actual_type}
                    
            if validation_status == "DATATYPE_MISMATCH":
                with open(self.config.STATUS_FILE, "w") as f:
                        f.write(f"Validation Status : {validation_status}")

                with open(self.config.STATUS_DATA_TYPE_FILE, "w") as f:
                        json.dump(mismatched_cols, f, indent=4)

            return mismatched_cols
        except Exception as e:
            raise e
        
    def get_validation_status(self) -> bool:

        with open(self.config.STATUS_FILE, "r") as f:
            status = f.read().split(" ")[-1]

        return status


        

In [9]:
config = ConfigurationManager()
data_validation_config = config.get_data_validation_config()

[2025-02-27 01:23:40,941: common:yaml file: config\config.yaml loaded successfully]
[2025-02-27 01:23:40,943: common:yaml file: params.yaml loaded successfully]
[2025-02-27 01:23:40,947: common:yaml file: schema.yaml loaded successfully]
[2025-02-27 01:23:40,949: common:created directory at artifacts]
[2025-02-27 01:23:40,949: common:created directory at artifacts/data_validation]


In [10]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(data_validation_config)
    data_validation.validate_all_columns()
    data_validation.validate_column_datatype()

except Exception as e:
    raise e


[2025-02-27 01:23:40,973: common:yaml file: config\config.yaml loaded successfully]
[2025-02-27 01:23:40,975: common:yaml file: params.yaml loaded successfully]
[2025-02-27 01:23:40,977: common:yaml file: schema.yaml loaded successfully]
[2025-02-27 01:23:40,980: common:created directory at artifacts]
[2025-02-27 01:23:40,980: common:created directory at artifacts/data_validation]
